In [ ]:
from os import listdir
from os.path import isfile, join
import os
import pandas as pd
import numpy as np
from collections import Counter

from collections import defaultdict

In [ ]:
files = [f for f in listdir("data") if isfile(join("data", f))]
if "index.json" in files:
    files.remove("index.json")
"index.json" in files

In [ ]:
dfs = []
for i in files:
    dfs.append(pd.read_csv(os.path.join("data", i)))

In [ ]:
data = pd.concat(dfs).reset_index()

In [ ]:
data.isna().mean()

Process address

In [ ]:
Counter([eval(i)[2].split()[0] if len(eval(i)) > 2 else None for i in data["address"] ]).most_common()

In [ ]:
def parse_address(address):
    try:
        City = address[0]
        District = address[1]
        Ward = address[2] if len(address) > 2  and any([i in address[2].lower() for i in ["xã", "phường", "thị trấn"]]) else "None"
        Street = address[3] if len(address) > 3 and any([i in address[3].lower() for i in ["đường", "phố"]]) else "None"
    except Exception as e:
        print(e)
        print(address)
        return {}
    return dict(City=City, District=District, Ward=Ward, Street=Street)

In [ ]:
df_address = pd.DataFrame([parse_address(eval(data["address"][i])) for i in range(len(data["address"]))])

In [ ]:
data = pd.concat([data, df_address], axis=1)

In [ ]:
data.head()

Area and price

In [ ]:
data["Area"] = data["Diện tích"].apply(lambda x: float(x.split()[0]) if not pd.isnull(x) else x)
data = data.drop("Diện tích", axis=1)
data["Area"].head()

In [ ]:
def parce_price(data):
    prices = []
    for idx, i in enumerate(data["Mức giá"]):
        try:
            
            if i.split()[1] not in ["tỷ", "triệu/m²", "triệu"] or i == "Thỏa thuận":
                prices.append(None)
            else:
                if "tỷ" in i:
                    prices.append(float(i.split()[0])*1e3)
                elif "triệu/m²" in i:
                    prices.append(float(i.split()[0])*data["Area"][idx])
                else:
                    prices.append(float(i.split()[0]))
        except Exception as e:
            print(e)
            print(i, type(i))
    return prices

In [ ]:
data["Price"] = parce_price(data)
#data["Price"] = data["Price"].astype("float")

In [ ]:
data.head()

Other features

In [ ]:
features = {"Số phòng ngủ":"Bedrooms", "Số toilet":"Toilets", "Hướng nhà":"House Direction", "Hướng ban công":"Balcony Direction", "Pháp lý":"Legits", 
            "Nội thất":"Furnitures", "Số tầng":"Floors", "Mặt tiền":"Facade", "Đường vào":"Entrance",
           "Ngày đăng":"Date",
           "Ngày hết hạn":"Expired Date"}


In [ ]:
data = data.rename(columns=features)

In [ ]:
data["Bedrooms"] = [i.split()[0] if not pd.isna(i) else "None" for i in data["Bedrooms"]]
data["Bedrooms"].head()

In [ ]:
data["Toilets"] = [i.split()[0] if not pd.isna(i) else "None" for i in data["Toilets"]]
data["Toilets"].head()

In [ ]:
data["House Direction"] = [i if not pd.isna(i) else "None" for i in data["House Direction"]]
data["House Direction"].head()

In [ ]:
data["Balcony Direction"] = [i if not pd.isna(i) else "None" for i in data["Balcony Direction"]]
data["Balcony Direction"].head()

In [ ]:
data["Facade"] = [i.split()[0] if not pd.isna(i) else "None" for i in data["Facade"]]
data["Facade"].head()

In [ ]:
data["Furnitures"] = [i if not pd.isna(i) else "" for i in data["Furnitures"]]
data["Furnitures"].head()

In [ ]:
data["Legits"] = [i if not pd.isna(i) else "None" for i in data["Legits"]]
data["Legits"].head()

In [ ]:
data["Floors"] = [i.split()[0] if not pd.isna(i) else "None" for i in data["Floors"]]
data["Floors"]

In [ ]:
data["Entrance"] = [i.split()[0] if not pd.isna(i) else "None" for i in data["Entrance"]]
data["Entrance"]

In [ ]:
data = data[data["District"] != "Hà Nội. 73m²* 5.4 tỷ. Nhà đẹp siêu vip"]
data = data[data["District"] != "Hà Nội. 76m²* 5T 5.95 tỷ. Nhà đẹp siêu vip"]

In [ ]:
data.columns

In [ ]:
features = ['House Direction',
       'Balcony Direction', 'Bedrooms', 'Toilets', 
        'Legits', 'Furnitures', 'Floors', 'Facade',
       'Entrance', 'City', 'District', 'Ward', 'Street', 'Area', 'Price']

In [ ]:
data[features]

In [ ]:
final_data = data[features]
final_data = final_data.dropna()

In [ ]:
final_data.to_csv("final_data.csv", index=False)

In [ ]:
price = price[price != 2.3e8]

In [ ]:
price.dropna().describe()

In [ ]:
price.value_counts()

In [ ]:
price.plot.box()

In [ ]:
(price[price < 1.2e4]/1e4).plot.box()

In [ ]:
(price[price < 1.2e4]/1e4).plot.hist(bins=200, figsize=(15,5), title="Historgram for quantile 75%")

In [ ]:
data.dropna()

In [ ]:
np.isnan(data["Số toilet"][3])
